In [1]:
#from openai import OpenAI
from g4f.client import Client
from db_utils import connect_to_db, execute_sql,query_database
import re

In [2]:
client = Client()

In [3]:
def generate_sql_from_nl(user_input):
    # mapping_table = load_mapping_from_csv()
    system_prompt = f''''
        1. 你是一個資料庫專家，能夠生成相應的SQL指令。 
        2. SQL的編寫請用全英文。
        3. 非常重要!返回sql語法即可，不用其他說明與文字。
        4. MYSQL code中請不要出現沒有的英文名稱。
        '''

    user_prompt = '''
        資料庫結構、表名稱、欄位名稱請參考以下內容:
        table_name,table_name_cn,column_name,column_name_cn
        customer,顧客,CUSTID,顧客ID
        customer,顧客,NAME,名稱
        customer,顧客,ADDRESS,地址
        customer,顧客,CITY,城市
        customer,顧客,STATE,州
        customer,顧客,ZIP,郵政編碼
        customer,顧客,AREA,區域
        customer,顧客,PHONE,電話
        customer,顧客,REPID,代表ID
        customer,顧客,CREDITLIMIT,信用額度
        dept,部門,DEPTNO,部門號
        dept,部門,DNAME,部門名稱
        dept,部門,LOC,位置
        emp,員工,EMPNO,員工號
        emp,員工,ENAME,員工姓名
        emp,員工,JOB,職位
        emp,員工,MGR,經理
        emp,員工,HIREDATE,入職日期
        emp,員工,SAL,工資
        emp,員工,COMM,佣金
        emp,員工,DEPTNO,部門號
        item,項目,ORDID,訂單ID
        item,項目,PRODID,產品ID
        item,項目,ACTUALPRICE,實際價格
        item,項目,QTY,數量
        item,項目,ITEMTOT,項目總計
        ord,訂單,ORDID,訂單ID
        ord,訂單,ORDERDATE,訂單日期
        ord,訂單,COMMPLAN,佣金計劃
        ord,訂單,CUSTID,顧客ID
        ord,訂單,SHIPDATE,發貨日期
        ord,訂單,TOTAL,總計
        price,價格,PRODID,產品ID
        price,價格,STDPRICE,標準價格
        price,價格,MINPRICE,最低價格
        price,價格,STARTDATE,開始日期
        price,價格,ENDDATE,結束日期
        product,產品,PRODID,產品ID
        product,產品,DESCRIP,描述
        salgrade,薪資等級,GRADE,等級
        salgrade,薪資等級,LOSAL,最低薪水
        salgrade,薪資等級,HISAL,最高薪水
                '''
    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input + user_prompt}
        ],
        max_tokens=150,
        temperature=0.1
    )
    sql_response = response.choices[0].message.content
    return sql_response


In [4]:
def extract_sql_query(sql_response): 
    match = re.search(r'SELECT.*', sql_response, re.IGNORECASE) 
    if match: 
        return match.group() 
    else: 
        raise ValueError("SQL查詢指令未找到")

In [5]:
if __name__ == "__main__":
    user_input ="請幫我查詢員工表內所有資訊"   #input("請輸入內容: ") 
    # 初次調用GPT判斷是否需要function calling 
    response_1 = client.chat.completions.create( model="gpt-4-turbo", 
                                                messages=[
                                                {"role": "system", 
                                                 "content":"請判斷使用者輸入是否需要連線到資料庫查詢，若需要請回答YES即可!若不需要則正常對話"},
                                                {"role": "user", "content": user_input}])
    
    print('===== 初次調用GPT判斷 =====')
    if 'YES' in response_1.choices[0].message.content:
        sql_response = generate_sql_from_nl(user_input) 
        sql_query = extract_sql_query(sql_response)
        print(f'生成的SQL指令: {sql_query}') 
        # 執行GPT生成的SQL查詢 
        result = query_database(sql_query)
        print(f'查詢結果: {result}')

    else: 
        # 一般對話處理
        print(response_1.choices[0].message.content )
         


New g4f version: 0.3.9.1 (current: 0.3.8.1) | pip install -U g4f
===== 初次調用GPT判斷 =====


C:\Users\TMP214\AppData\Local\Programs\Python\Python39\lib\asyncio\events.py:80: UserWarning: Curlm alread closed! quitting from process_data
  self._context.run(self._callback, *self._args)


生成的SQL指令: SELECT * FROM emp;
連線成功
查詢結果: [{"EMPNO": 7369, "ENAME": "SMITH", "JOB": "CLERK", "MGR": 7902, "HIREDATE": "2010-12-17T00:00:00", "SAL": 800.0, "COMM": null, "DEPTNO": 20}, {"EMPNO": 7499, "ENAME": "ALLEN", "JOB": "SALESMAN", "MGR": 7698, "HIREDATE": "2011-02-20T00:00:00", "SAL": 1600.0, "COMM": 300.0, "DEPTNO": 30}, {"EMPNO": 7521, "ENAME": "WARD", "JOB": "SALESMAN", "MGR": 7698, "HIREDATE": "2011-02-22T00:00:00", "SAL": 1250.0, "COMM": 500.0, "DEPTNO": 30}, {"EMPNO": 7566, "ENAME": "JONES", "JOB": "MANAGER", "MGR": 7839, "HIREDATE": "2011-04-02T00:00:00", "SAL": 2975.0, "COMM": null, "DEPTNO": 20}, {"EMPNO": 7654, "ENAME": "MARTIN", "JOB": "SALESMAN", "MGR": 7698, "HIREDATE": "2011-09-28T00:00:00", "SAL": 1250.0, "COMM": 1400.0, "DEPTNO": 30}, {"EMPNO": 7698, "ENAME": "BLAKE", "JOB": "MANAGER", "MGR": 7839, "HIREDATE": "2011-05-01T00:00:00", "SAL": 2850.0, "COMM": null, "DEPTNO": 30}, {"EMPNO": 7782, "ENAME": "CLARK", "JOB": "MANAGER", "MGR": 7839, "HIREDATE": "2011-06-09T00